<a href="https://colab.research.google.com/github/Pranshu244/Startup-Idea-Research-Assistant/blob/main/Chatbot%20_Code/Startup_Idea_Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain

In [ ]:
!pip install -U langchain-community langchain-core pypdf

In [ ]:
!pip install -U langchain-google-genai faiss-cpu tiktoken python-dotenv

In [ ]:
!pip install -U langchain-text-splitters

In [ ]:
!pip install -U langchainhub langchain-community sentence-transformers

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "INSERT_YOUR_API_KEY_OF_gemini-2.5-flash"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("AFTER UPLOADING YOUR DOCUMENT WRITE IT'S NAME (eg:AI_Startup_Document.pdf)")
docs=loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    )
chunks=splitter.split_documents(docs)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from langchain_community.vectorstores import FAISS
vector_store=FAISS.from_documents(chunks,embedding)

In [ ]:
retriever=vector_store.as_retriever(search_type="mmr",search_kwargs={"k":5,"lambda_mult": 0.7})

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt=PromptTemplate(
    template="""
      You are a helpful research assistant who responds to the user as a friendly companion.
      Your job is to compare the user's startup idea (provided in the context) with existing companies.
      You will help improve their idea and offer thoughtful, constructive suggestions. Stay focused on startup-related topics.
      If the user tries to go off-topic, politely respond that your role is to assist only with startup research and idea development.
      You can discuss startups, but not unrelated subjects. Always begin your response directly with the answer — do not include greetings
      as it is a continued chat but if user greet you then greet him then don ot start the answer automatically. Listen to his query then answer.
      Never abuse and never say any inappropirate terms , be polite, no bad language and misbehave even if user aangry.

      {context}
      Question:{question}
    """,
    input_variables=["context","question"]
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(
                    model="gemini-2.5-flash",
                    temperature=0.5,
                    timeout=30
                    )

In [ ]:
from langchain_core.runnables import RunnableLambda,RunnablePassthrough,RunnableParallel
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [ ]:
def format_docs(retrieved_docs):
  context_text="\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain= RunnableParallel({
    "context" : retriever|RunnableLambda(format_docs),
    "question" : RunnablePassthrough()
})

In [ ]:
main_chain=parallel_chain|prompt|model|parser

In [ ]:
import gradio as gr

def respond(message, history):
    try:
        reply = main_chain.invoke(message)
    except Exception as e:
        reply = f"⚠️ Error: {type(e).__name__}: {e}"
    history.append((message, reply))
    return history, history

with gr.Blocks(css="""
body {background: linear-gradient(135deg, #ffcc70, #ff8c42);}
h1 {text-align: center; font-style: italic; font-weight: bold; color: #333;}
#chatbot {border-radius: 12px; background-color: #fffaf0;}
#msg-box {width: 75%; margin: auto;}
button {background-color: #ff8c42 !important; color: white !important; border-radius: 8px;}
label {font-weight: bold; color: #ff5722; font-size: 16px;}
""") as demo:
    gr.HTML("<h1>🚀 Startup Idea Research Assistant</h1>")
    chatbot = gr.Chatbot(label="💬 Chatbox", height=500, show_copy_button=True, elem_id="chatbot")
    msg = gr.Textbox(label="✍️ Your Question", placeholder="Type your question here...", elem_id="msg-box")
    with gr.Row():
        submit = gr.Button("Submit")
        clear = gr.Button("Clear")
    submit.click(respond, [msg, chatbot], [chatbot, chatbot])
    msg.submit(respond, [msg, chatbot], [chatbot, chatbot])
    clear.click(lambda: [], None, chatbot)

demo.launch(share=True, debug=True)
